In [1]:
from dataReadFromDisk import *

In [2]:
df = load_df_from_csv()
ndf = df[0:1]
ndf

DataFrame loaded from: E:\\Notebooks\\osu\processed-metadata\processed-metadata.csv


,audio,split,folder,beatmapset_id,beatmap_id,approved,total_length,hit_length,version,file_md5,...,difficultyrating,StackLeniency,DistanceSpacing,BeatDivisor,HPDrainRate,CircleSize,OverallDifficulty,ApproachRate,SliderTickRate,SliderMultiplier
0,0013ddfd8bd55fdccc0b253e313b7a60,train,6fc07475587f45a69547eb7f9b51a0c3,1327294,2847694,1,305,303,Insane,0013ddfd8bd55fdccc0b253e313b7a60,...,5.159,0.6,1.2,4,6,4,8,9,1,2.2


In [3]:
# all_beatmaps, all_audios = load_all_beatmaps_and_audios_from_df(ndf)

In [4]:
# all_audios[0][0]

In [5]:
data = load_all_beatmaps_from_df(ndf)

In [6]:
for x in data[0]:
    print(x)

[  37.            201.           2140.              6.
    0.              2.             37.            201.
    1.             38.4999988251    2.              3.
    2.              0.              0.              0.
    0.              0.           2278.           2140.
  142.8571428571    4.              1.              1.
   35.              0.              0.           2140.
  555.5555555556    4.              1.              1.
   35.              1.              0.          ]
[ 130.            188.           2278.              6.
    0.              2.            130.            188.
    1.             38.4999988251    0.              3.
    0.              0.              0.              0.
    0.              2.           2416.           2140.
  142.8571428571    4.              1.              1.
   35.              0.              0.           2140.
  555.5555555556    4.              1.              1.
   35.              1.              0.          ]
[ 175.            14

[ 196.             77.           9640.              2.
    0.              3.            196.             77.
    1.             25.6666658834    2.              2.
    2.              0.              0.              0.
    0.              0.           9738.           9640.
  142.8571428571    4.              1.              1.
   35.              0.              0.           9640.
  588.2352941176    4.              1.              1.
   35.              1.              0.          ]
[ 166.             71.           9738.              2.
    0.              3.            166.             71.
    1.             25.6666658834    0.              3.
    0.              0.              0.              0.
    0.              1.           9836.           9640.
  142.8571428571    4.              1.              1.
   35.              0.              0.           9640.
  588.2352941176    4.              1.              1.
   35.              1.              0.          ]
[ 112.             4

In [7]:
processed_audio, processed_beatmap = process_audio_and_beatmap_for_model("0013ddfd8bd55fdccc0b253e313b7a60")

In [8]:
all_beatmaps, all_audios = load_all_beatmaps_and_audios_from_df(ndf)

1/1


In [9]:
all_audios = np.array(all_audios)
all_audios.shape
all_audios[0][0].shape


(56, 13)

In [16]:
len(processed_audio)

476